In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.ensemble import *
from datetime import datetime
from tensorflow.keras.callbacks import *

In [2]:
df = pd.read_csv("./Datasets/GW3/soccer-spi/spi_matches.csv")

In [3]:
df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def dateconv (dates):
  date = datetime.strptime(dates,'%Y-%m-%d') 
  return date

df['date'] = df['date'].apply(lambda x : dateconv(x))

In [74]:
last_played = datetime(2020,9,25)
GW_end = datetime(2020,9,29)
pl = df[df['league']== "Barclays Premier League"]
old = pl[pl['date']<last_played]
new = pl[pl['date']>last_played]
new = new[new['date'] < GW_end]

In [75]:
teams = pd.get_dummies(new[['team1','team2']])
teamsold = pd.get_dummies(old[['team1','team2']])

In [76]:
new = pd.concat([new,teams], axis=1)
old = pd.concat([old,teamsold], axis=1)

In [77]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 34586 to 34823
Data columns (total 43 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   season                          10 non-null     int64         
 1   date                            10 non-null     datetime64[ns]
 2   league_id                       10 non-null     int64         
 3   league                          10 non-null     object        
 4   team1                           10 non-null     object        
 5   team2                           10 non-null     object        
 6   spi1                            10 non-null     float64       
 7   spi2                            10 non-null     float64       
 8   prob1                           10 non-null     float64       
 9   prob2                           10 non-null     float64       
 10  probtie                         10 non-null     float64       
 11  p

In [49]:
old_teams = old.columns.tolist()
new_teams = new.columns.tolist()

In [50]:
dropped_columns_old = []
dropped_columns_new = []
for column in old_teams:
    if column not in new_teams:
        dropped_columns_old.append(column)
for column in new_teams:
    if column not in old_teams:
        dropped_columns_new.append(column)

In [51]:
dropped_columns_new

[]

In [78]:
X_train = old[['season', 'spi1', 'spi2', 'score1','score2','prob1', 'prob2', 'probtie', 'proj_score1',
       'proj_score2', 'team1_Brighton and Hove Albion', 'team1_Burnley',
       'team1_Crystal Palace', 'team1_Fulham', 'team1_Liverpool',
       'team1_Manchester City', 'team1_Sheffield United',
       'team1_Tottenham Hotspur', 'team1_West Bromwich Albion',
       'team1_West Ham United', 'team2_Arsenal', 'team2_Aston Villa',
       'team2_Chelsea', 'team2_Everton', 'team2_Leeds United',
       'team2_Leicester City', 'team2_Manchester United', 'team2_Newcastle',
       'team2_Southampton', 'team2_Wolverhampton']]
X_test = new[['season', 'spi1', 'spi2','prob1', 'prob2', 'probtie', 'proj_score1',
       'proj_score2','team1_Brighton and Hove Albion', 'team1_Burnley',
       'team1_Crystal Palace', 'team1_Fulham', 'team1_Liverpool',
       'team1_Manchester City', 'team1_Sheffield United',
       'team1_Tottenham Hotspur', 'team1_West Bromwich Albion',
       'team1_West Ham United', 'team2_Arsenal', 'team2_Aston Villa',
       'team2_Chelsea', 'team2_Everton', 'team2_Leeds United',
       'team2_Leicester City', 'team2_Manchester United', 'team2_Newcastle',
       'team2_Southampton', 'team2_Wolverhampton']]

In [81]:
X_train.dropna(inplace=True)

In [83]:
X_test.dropna(inplace=True)

In [85]:
lm = LinearRegression()
lm.fit(X_train.drop(['score1','score2'],axis=1),X_train[['score1','score2']])
predictions  = lm.predict(X_test)
preds = []
for num in predictions:
  score1 = np.round(num[0])
  score2 = np.round(num[1])
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Predicted Score 1'] = score1
new['Predicted Score 2'] = score2
new.to_excel("ProjectionsPLGW3_LR.xlsx")

10
10


In [90]:
tX_train = X_train.drop(['score1','score2'],axis=1).values
tX_test =X_test.values
y_train = X_train[['score1','score2']].values

In [91]:
scaler = MinMaxScaler()
tX_train = scaler.fit_transform(tX_train)
tX_test = scaler.transform(tX_test)
tX_train.shape

(1538, 28)

In [92]:
def two_to_three(array):
    array = np.reshape(array, array.shape+(1,))
    return array

In [93]:
tX_test = two_to_three(tX_test)
tX_train = two_to_three(tX_train)
print(tX_test.shape)
print(tX_train.shape)

(10, 28, 1)
(1538, 28, 1)


In [114]:
model = Sequential()

In [115]:

model.add(SimpleRNN(46, return_sequences=True))
model.add(SimpleRNN(46, return_sequences=True))
model.add(SimpleRNN(46))
model.add(Dense(92, activation='relu'))
model.add(Dense(2))


In [116]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [117]:
early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=25)

In [118]:
model.fit(tX_train,y_train,epochs=60)

Train on 1538 samples
Epoch 1/60
1538/1538 [==============================] - 4s 3ms/sample - loss: 1.5709 - accuracy: 0.6853
Epoch 2/60
1538/1538 [==============================] - 1s 861us/sample - loss: 1.3719 - accuracy: 0.6905
Epoch 3/60
1538/1538 [==============================] - 2s 2ms/sample - loss: 1.3269 - accuracy: 0.7165
Epoch 4/60
1538/1538 [==============================] - 3s 2ms/sample - loss: 1.3426 - accuracy: 0.7035
Epoch 5/60
1538/1538 [==============================] - 2s 1ms/sample - loss: 1.3410 - accuracy: 0.7087
Epoch 6/60
1538/1538 [==============================] - 1s 913us/sample - loss: 1.3225 - accuracy: 0.7224
Epoch 7/60
1538/1538 [==============================] - 2s 1ms/sample - loss: 1.3119 - accuracy: 0.7068
Epoch 8/60
1538/1538 [==============================] - 1s 923us/sample - loss: 1.3216 - accuracy: 0.7159
Epoch 9/60
1538/1538 [==============================] - 2s 991us/sample - loss: 1.3158 - accuracy: 0.7009
Epoch 10/60
1538/1538 [===========

In [119]:
predictions  = model.predict(tX_test)
preds = []
for num in predictions:
  score1 = np.round(num[0]) if num[0]>=0 else 0
  score2 = np.round(num[1]) if num[1]>=0 else 0
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Predicted Score 1'] = score1
new['Predicted Score 2'] = score2
new.to_excel("ProjectionsPLGW3_TF.xlsx")

10
10
